# NLP - Project 1
## Rinehart Analysis
**Team**: *Jean Merlet, Konstantinos Georgiou, Matt Lane*

Check out the **[README](https://github.com/NLPaladins/rinehartAnalysis/blob/main/README.md)**


Or the current **[TODO](https://github.com/NLPaladins/rinehartAnalysis/blob/main/TODO.md)** list.

In [1]:
# Import Jupyter Widgets
import os
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display

In [2]:
# Clone the repository if you're in Google Collab
def clone_project(is_collab: bool = False):
    print("Cloning Project..")
    !git clone https://github.com/NLPaladins/rinehartAnalysis.git
    print("Project cloned.")
       
print("Clone project?")
print("(If you do this you will ovewrite local changes on other files e.g. configs)")
print("Not needed if you're not on Google Collab")
btn = widgets.Button(description="Yes, clone")
btn.on_click(clone_project)
display(btn)

Clone project?
(If you do this you will ovewrite local changes on other files e.g. configs)
Not needed if you're not on Google Collab


Button(description='Yes, clone', style=ButtonStyle())

In [3]:
# Clone the repository if you're in Google Collab
def change_dir(is_collab: bool = False):
    try:
        print("Changing dir..")
        os.chdir('/content/rinehartAnalysis')
        print('done')
        print("Current dir:")
        print(os.getcwd())
        print("Dir Contents:")
        print(os.listdir())
    except Exception:
        print("Error: Project not cloned")
       
print("Are you on Google Collab?")
btn = widgets.Button(description="Yes")
btn.on_click(change_dir)
display(btn)

Are you on Google Collab?


Button(description='Yes', style=ButtonStyle())

### At any point, to save changes
click **File > Save a copy on Gihtub**

### Now go to files on the left, and open:
- rinehartAnalysis/confs/proj_1.yml

(Ctr+s) to save changes

## Load Libraries and setup

In [4]:
import traceback
import argparse
from importlib import reload as reload_lib
from pprint import pprint
import numpy as np

# Custom libs
from nlp_libs import Configuration, ColorizedLogger, ProcessedBook
# Import this way the libs you want to dynamically change and reload 
# import nlp_libs.books.processed_book as books_lib # Comment out until Class is finalized


### Libraries Overview
All the libraries are located under *"\<project root>/nlp_libs"*
- ***ProcessedBook***: Loc: **books/processed_book.py**, Desc: *Book Pre-processor*
- ***Configuration***: Loc: **configuration/configuration.py**, Desc: *Configuration Loader*
- ***ColorizedLogger***: Loc: **fancy_logger/colorized_logger.py**, Desc: *Logger with formatted text capabilities*

In [5]:
# The path of configuration and log save path
config_path = "confs/proj_1.yml"  # Open files > confs > proj_1.yml to edit temporalily. Commit to save permanently
# !cat "$config_path"
log_path = "logs/proj_1.log"  # Open files > logs > proj_1.log to debug logs of previous runs

In [6]:
# The logger
logger = ColorizedLogger(logger_name='Notebook', color='cyan')
ColorizedLogger.setup_logger(log_path=log_path, debug=False, clear_log=True)

2021-09-22 16:43:39 FancyLogger  INFO     Logger is set. Log file path: /Users/96v/Documents/DSE/nlp/rinehartAnalysis/logs/proj_1.log


In [7]:
# Load the configuration
conf = Configuration(config_src=config_path)
# Get the books dict
books = conf.get_config('data_loader')['config']['books']
pprint(books)  # Pretty print the books dict

2021-09-22 16:43:39 Config       INFO     Configuration file loaded successfully from path: /Users/96v/Documents/DSE/nlp/rinehartAnalysis/confs/proj_1.yml
2021-09-22 16:43:39 Config       INFO     Configuration Tag: proj1


{'Oh,_Well,_You_Know_How_Women_Are!': {'crime_type': 'example',
                                       'detectives': ['man1', 'man2'],
                                       'suspects': ['man3', 'man4'],
                                       'url': 'https://www.gutenberg.org/cache/epub/24259/pg24259.txt'},
 'The_Breaking_Point': {'crime_type': 'example',
                        'detectives': ['man1', 'man2'],
                        'suspects': ['man3', 'man4'],
                        'url': 'https://www.gutenberg.org/files/1601/1601-0.txt'},
 'The_Circular_Staircase': {'crime_type': 'example',
                            'detectives': ['Rachel Innes'],
                            'suspects': ['Liddy',
                                         'Halsey',
                                         'Gertrude',
                                         'Paul Armstrong',
                                         'Doctor Walker',
                                         'Louise Armstrong',
    

## Exploration

In [17]:
r'someString\nanotherstring'.split(r'\n')

['someString', 'anotherstring']

In [8]:
import urllib.request
import re
from typing import *


class ProcessedBook:
    num_map = [(1000, 'M'), (900, 'CM'), (500, 'D'), (400, 'CD'),
               (100, 'C'), (90, 'XC'), (50, 'L'), (40, 'XL'),
               (10, 'X'), (9, 'IX'), (5, 'V'), (4, 'IV'), (1, 'I')]
    title: str
    url: str
    detectives: List[str]
    suspects: List[str]
    crime_type: str

    def __init__(self, title: str, metadata: Dict, make_lower: bool = True):
        """
        raw holds the books as a single string.
        clean holds the books as a list of lowercase lines starting
        from the first chapter and ending with the last sentence.
        """
        self.title = title
        self.url = metadata['url']
        self.detectives = metadata['detectives']
        self.suspects = metadata['suspects']
        self.crime_type = metadata['crime_type']
        self.raw = self.read_book_from_proj_gut(self.url)
        if make_lower:
            lines = self.raw.lower()
        else:
            lines = self.raw
        loins = re.sub(r'\r\n', r'\n',staircase.raw)
        loins = re.findall(r'.*\n', loins)
        self.lines = self.clean_lines(lines=lines)
        self.clean = self.get_clean_book(make_lower=make_lower)

    @staticmethod
    def read_book_from_proj_gut(book_url: str) -> str:
        req = urllib.request.Request(book_url)
        client = urllib.request.urlopen(req)
        page = client.read()
        return page.decode('utf-8')

    def get_clean_book(self, make_lower: bool = True) -> List[str]:
        if make_lower:
            lines = self.raw.lower()
        else:
            lines = self.raw
        lines = lines.replace('\r\n', '\n').split('\n')
        lines = self.clean_lines(lines)
        chapters = self.lines_to_chapters(lines)
        return chapters

    def clean_lines(self, lines: List[str]) -> List[str]:
        clean_lines = []
        start = False
        for line in lines:
            if re.match(r'^chapter i\.', line):
                clean_lines.append(line)
                start = True
                continue
            if not start:
                continue
            if re.match(r'^\*\*\* end of the project gutenberg ebook', line):
                break
            if self.pass_clean_filter(line):
                clean_lines.append(line)
        return clean_lines

    @staticmethod
    def lines_to_chapters(lines: List[str]) -> List[str]:
        chapters = []
        sentences = []
        current_sent = ''
        for i, line in enumerate(lines):
            # add chapter as 1st sentence
            if re.match(r'^chapter [ivxlcdm]+\.$', line):
                if sentences:
                    chapters.append(sentences)
                sentences = [line]
                add_chapter_title = True
                continue
            # add chapter title as 2nd sentence
            elif add_chapter_title:
                sentences.append(line)
                add_chapter_title = False
                continue
            sents = re.findall(r' *((?:mr\.|mrs.|[^\.\?!])*)(?<!mr)(?<!mrs)[\.\?!]', line)
            # if no sentence end is detected
            if not sents:
                if current_sent == '':
                    current_sent = line
                else:
                    current_sent += ' ' + line
            # if at least one sentence end is detected
            else:
                for group in sents:
                    if current_sent != '':
                        current_sent += ' ' + group
                        sentences.append(current_sent)
                    else:
                        sentences.append(group)
                    current_sent = ''
                # set the next sentence to its start if there is one
                sent_end = re.search(r'(?<!mr)(?<!mrs)[\.\?!] ((?:mr\.|mrs\.|[^\.\?!])*)$', line)
                if sent_end is not None:
                    current_sent = sent_end.groups()[0]
        return chapters

    @staticmethod
    def pass_clean_filter(line: str) -> bool:
        # removing the illustration lines and empty lines
        # can add other filters here as needed
        if line == '' or re.match(r'illustration:|\[illustration\]', line):
            return False
        else:
            return True

    def get_chapter(self, chapter: int) -> str:
        return self.clean[chapter - 1]


In [9]:
# Load Circlular Staircase
title, metadata = list(books.items())[0]
staircase = ProcessedBook(title=title, metadata=metadata)

In [10]:
logger.info(f'The raw length of this book as a string is {len(staircase.raw)}')
logger.info(f'This book has {len(staircase.clean)} chapters\n')
for i, chapter in enumerate(staircase.clean):
  if i == 5: break
  logger.info(f'{chapter[0]} - {chapter[1]}')
  logger.info(f'There are {len(chapter)} sentences in this chapter.')
  num_words = []
  for sent in chapter:
    num_words.append(len(sent.split(' ')))
  avg_words = np.mean(num_words)
  logger.info(f'The average sentence length in this chapter is {avg_words} words\n')

2021-09-22 16:43:41 Notebook     INFO     The raw length of this book as a string is 410135
2021-09-22 16:43:41 Notebook     INFO     This book has 33 chapters

2021-09-22 16:43:41 Notebook     INFO     chapter i. - i take a country house
2021-09-22 16:43:41 Notebook     INFO     There are 117 sentences in this chapter.
2021-09-22 16:43:41 Notebook     INFO     The average sentence length in this chapter is 21.307692307692307 words

2021-09-22 16:43:41 Notebook     INFO     chapter ii. - a link cuff-button
2021-09-22 16:43:41 Notebook     INFO     There are 142 sentences in this chapter.
2021-09-22 16:43:41 Notebook     INFO     The average sentence length in this chapter is 16.260563380281692 words

2021-09-22 16:43:41 Notebook     INFO     chapter iii. - mr. john bailey appears
2021-09-22 16:43:41 Notebook     INFO     There are 104 sentences in this chapter.
2021-09-22 16:43:41 Notebook     INFO     The average sentence length in this chapter is 14.663461538461538 words

2021-09-22 

In [11]:
for i, sent in enumerate(staircase.get_chapter(33)):
  if i == 10: break
  logger.info(sent)

2021-09-22 16:43:41 Notebook     INFO     chapter xxxiii.
2021-09-22 16:43:41 Notebook     INFO     at the foot of the stairs
2021-09-22 16:43:41 Notebook     INFO     as i drove rapidly up to the house from casanova station in the hack, i saw the detective burns loitering across the street from the walker place
2021-09-22 16:43:41 Notebook     INFO     so jamieson was putting the screws on—lightly now, but ready to give them a twist or two, i felt certain, very soon
2021-09-22 16:43:41 Notebook     INFO     the house was quiet
2021-09-22 16:43:41 Notebook     INFO     two steps of the circular staircase had been pried off, without result, and beyond a second message from gertrude, that halsey insisted on coming home and they would arrive that night, there was nothing new
2021-09-22 16:43:41 Notebook     INFO     mr. jamieson, having failed to locate the secret room, had gone to the village
2021-09-22 16:43:41 Notebook     INFO     i learned afterwards that he called at doctor walker’s

In [12]:
# Load Lower Ten
title, metadata = list(books.items())[0]
lower_ten = ProcessedBook(title=title, metadata=metadata)

In [13]:
logger.info(f'The raw length of this book as a string is {len(lower_ten.raw)}')
logger.info(f'This book has {len(lower_ten.clean)} chapters\n')
for i, chapter in enumerate(lower_ten.clean):
  if i == 5: break
  logger.info(f'{chapter[0]} - {chapter[1]}')
  logger.info(f'There are {len(chapter)} sentences in this chapter.')
  num_words = []
  for sent in chapter:
    num_words.append(len(sent.split(' ')))
  avg_words = np.mean(num_words)
  logger.info(f'The average sentence length in this chapter is {avg_words} words\n')

2021-09-22 16:43:43 Notebook     INFO     The raw length of this book as a string is 410135
2021-09-22 16:43:43 Notebook     INFO     This book has 33 chapters

2021-09-22 16:43:43 Notebook     INFO     chapter i. - i take a country house
2021-09-22 16:43:43 Notebook     INFO     There are 117 sentences in this chapter.
2021-09-22 16:43:43 Notebook     INFO     The average sentence length in this chapter is 21.307692307692307 words

2021-09-22 16:43:43 Notebook     INFO     chapter ii. - a link cuff-button
2021-09-22 16:43:43 Notebook     INFO     There are 142 sentences in this chapter.
2021-09-22 16:43:43 Notebook     INFO     The average sentence length in this chapter is 16.260563380281692 words

2021-09-22 16:43:43 Notebook     INFO     chapter iii. - mr. john bailey appears
2021-09-22 16:43:43 Notebook     INFO     There are 104 sentences in this chapter.
2021-09-22 16:43:43 Notebook     INFO     The average sentence length in this chapter is 14.663461538461538 words

2021-09-22 

In [14]:
for i, sent in enumerate(lower_ten.get_chapter(15)):
  if i == 10: 
    break
  logger.info(sent)

2021-09-22 16:43:43 Notebook     INFO     chapter xv.
2021-09-22 16:43:43 Notebook     INFO     liddy gives the alarm
2021-09-22 16:43:43 Notebook     INFO     the next day, friday, gertrude broke the news of her stepfather’s death to louise
2021-09-22 16:43:43 Notebook     INFO     she did it as gently as she could, telling her first that he was very ill, and finally that he was dead
2021-09-22 16:43:43 Notebook     INFO     louise received the news in the most unexpected manner, and when gertrude came out to tell me how she had stood it, i think she was almost shocked
2021-09-22 16:43:43 Notebook     INFO     “she just lay and stared at me, aunt ray,” she said
2021-09-22 16:43:43 Notebook     INFO     “do you know, i believe she is glad, glad
2021-09-22 16:43:43 Notebook     INFO     and she is too honest to pretend anything else
2021-09-22 16:43:43 Notebook     INFO     what sort of man was mr. paul armstrong, anyhow
2021-09-22 16:43:43 Notebook     INFO     “he was a bully as well 

In [15]:
# Example of running it on all the books
processed_books = {}
for title, metadata in books.items():
  logger.nl()
  logger.info(f"Book: {title}", color='yellow', attrs=['underline'])
  current_book = ProcessedBook(title=title, metadata=metadata)
  # Raw length
  logger.info(f'The raw length of this book as a string is {len(current_book.raw)}')
  # Number of chapters
  logger.info(f'This book has {len(current_book.clean)} chapters\n')
  # Sententences per chapter
  for i, chapter in enumerate(current_book.clean):
    if i == 5: 
      break
    logger.info(f'{chapter[0]} - {chapter[1]}')
    logger.info(f'There are {len(chapter)} sentences in this chapter.')
    num_words = []
    for sent in chapter:
      num_words.append(len(sent.split(' ')))
    avg_words = np.mean(num_words)
    logger.info(f'The average sentence length in this chapter is {avg_words} words\n')
  # Chapter 15
  for i, sent in enumerate(lower_ten.get_chapter(15)):
    if i == 10: break
    logger.info(sent)


2021-09-22 16:43:43 Notebook     INFO     Book: The_Circular_Staircase
2021-09-22 16:43:44 Notebook     INFO     The raw length of this book as a string is 410135
2021-09-22 16:43:44 Notebook     INFO     This book has 33 chapters

2021-09-22 16:43:44 Notebook     INFO     chapter i. - i take a country house
2021-09-22 16:43:44 Notebook     INFO     There are 117 sentences in this chapter.
2021-09-22 16:43:44 Notebook     INFO     The average sentence length in this chapter is 21.307692307692307 words

2021-09-22 16:43:44 Notebook     INFO     chapter ii. - a link cuff-button
2021-09-22 16:43:44 Notebook     INFO     There are 142 sentences in this chapter.
2021-09-22 16:43:44 Notebook     INFO     The average sentence length in this chapter is 16.260563380281692 words

2021-09-22 16:43:44 Notebook     INFO     chapter iii. - mr. john bailey appears
2021-09-22 16:43:44 Notebook     INFO     There are 104 sentences in this chapter.
2021-09-22 16:43:44 Notebook     INFO     The average 

2021-09-22 16:43:48 Notebook     INFO     “she just lay and stared at me, aunt ray,” she said
2021-09-22 16:43:48 Notebook     INFO     “do you know, i believe she is glad, glad
2021-09-22 16:43:48 Notebook     INFO     and she is too honest to pretend anything else
2021-09-22 16:43:48 Notebook     INFO     what sort of man was mr. paul armstrong, anyhow
2021-09-22 16:43:48 Notebook     INFO     “he was a bully as well as a rascal, gertrude,” i said

2021-09-22 16:43:48 Notebook     INFO     Book: The_Window_at_the_White_Cat
2021-09-22 16:43:48 Notebook     INFO     The raw length of this book as a string is 87360
2021-09-22 16:43:48 Notebook     INFO     This book has 0 chapters

2021-09-22 16:43:48 Notebook     INFO     chapter xv.
2021-09-22 16:43:48 Notebook     INFO     liddy gives the alarm
2021-09-22 16:43:48 Notebook     INFO     the next day, friday, gertrude broke the news of her stepfather’s death to louise
2021-09-22 16:43:48 Notebook     INFO     she did it as gently as sh

In [23]:
lines = staircase.raw.replace('\r\n', '\n').split('\n')

In [21]:
loins = re.sub(r'\r\n', r'\n',staircase.raw)
loins = re.findall(r'.*\n', loins)

In [31]:
for lineIdx in range(len(loins)): 
    loins[lineIdx] = re.sub(r'\n', '', loins[lineIdx])